In [59]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import pandas as pd
#from openai import OpenAI
import openai
import re
import json


openai.api_key = "ca72e43668824cd090e6ea210fe3cd3b"
openai.api_base = "https://hlt-nlp.openai.azure.com/" # your endpoint should look like the following
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
deployment_name='gpt-35-turbo'

In [60]:
def create_foilGPT(caption):
    response = openai.ChatCompletion.create(
        engine= deployment_name, # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
        messages=[
            {"role": "system", "content": f"You are an assistant designed to create foils for video captions." 
            f"Users will paste in a string of text a sentence C that is the caption of a video. Based on C, you must create only one foil F for that caption. Don't type any other information." 
            f"Remember: A foil differs from the caption for only one element!"},
            {"role": "user", "content": f"C: {caption}"}
        ]
        
    )
    output= response

    while output['choices'][0]['finish_reason'] == 'content_filter':
        response = openai.ChatCompletion.create(
            engine= deployment_name, # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
            messages=[
                {"role": "system", "content": f"You are an assistant designed to create foils for video captions." 
                f"Users will paste in a string of text a sentence C that is the caption of a video. Based on C, you must create only one foil F for that caption. Don't type any other information." 
                f"Remember: A foil differs from the caption for only one element!"
                #f"Ensure that the generated content is appropriate and follows OpenAI's usage policies."
                },
                {"role": "user", "content": f"C: {caption}"}
            ]
        
        )
        output= response

    

    while not re.findall('^F: .*|^Foil: .*', output['choices'][0]['message']['content']):
        response = openai.ChatCompletion.create(
            engine= deployment_name, # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
            messages=[
                {"role": "system", "content": f"You are an assistant designed to create foils for video captions." 
                f"Users will paste in a string of text a sentence C that is the caption of a video. Based on C, you must create only one foil F for that caption. Don't type any other information." 
                f"Remember: A foil differs from the caption for only one element!"},
                {"role": "user", "content": f"C: {caption}"}
            ]
        
        )
        output= response



    return output['choices'][0]['message']['content']




In [67]:
r = create_foilGPT(df.iloc[335,5])
r.strip('F: ')


"The girls' activities are uncertain."

In [69]:
r = r.strip('F: ')
r

"The girls' activities are uncertain."

In [70]:
df.iloc[335, 6] = r

In [71]:
df.to_csv('DF_pilot_withCaptionFoils.csv')

In [61]:
#load the df daaa cui prendere la caption e poi ci sovrascrivo i nuovi foil
df = pd.read_csv('/home/dtesta/MyProjects/Benchmark_PilotTest/DF_pilot_withCaptionFoils.csv', index_col = 0)
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,The video shows the girl using a phone to tak...
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,The people can protect themselves during an e...
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,The boy's experience with the computer is unc...
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,The weather outside is not audible.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,The girl is happy and dissatisfied with the w...
...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The boy with the hammer gets back up.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the axe manages to hit the boy w...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","After being hit by the axe, the boy with the ..."
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The girls are playing.


In [65]:
df.iloc[335,5]

"The boys' activities are uncertain."

In [220]:
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,The video shows the girl using a phone to mak...
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,The people can protect themselves during an e...
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,The girl's experience with the computer is ce...
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,The weather outside is visible.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,The girl is unhappy and dissatisfied with the...
...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The boy with the hammer gets back up.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the axe manages to hit the boy w...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","After being hit by the axe, the boy with the ..."
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The girls are playing.


In [336]:
#GPT generating foils based on captions
for index, row in df.iterrows():

    caption = row['Caption']
    foil = create_foilGPT(caption)
    df.loc[index, 'Foil'] = foil
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
2

KeyboardInterrupt: 

In [38]:
#clean up noise from caption_outputs
for index, row in df.iterrows():

    cleaned_foil = re.sub(r'\w* foil:|Foil\d*:|"', '', row['Foil'])  
    df.loc[index, 'Foil'] = cleaned_foil

In [338]:
for index, row in df.iterrows():

    print(row['Foil'])

F: The video shows the girl using a phone to take a picture.
F: The people can protect themselves during an earthquake by going on top of desks.
F: The boy's experience with the computer is uncertain.
F: The weather outside is not audible.
F: The girl is happy and dissatisfied with the woman's claim.
F: The hand at the end of the video most likely belongs to the person who is visible in the video.
F: A girl walks around the room while reading.
F: The girl is frowning at the end of the video.
F: The girl unsuccessfully operates the computer after receiving advice from the teacher.
F: This video takes place in a chemistry laboratory. (replacing "school" with "chemistry")
F: The video was recorded by something.
F: The class does not have any plants.
F: Other columns could be behind the desk.
F: The student is happy.
F: People are standing in the desks on the left.
F: The student moves around the teacher.
F: The student unsuccessfully performs the exercise after receiving instructions from

In [340]:
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,F: The video shows the girl using a phone to t...
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,F: The people can protect themselves during an...
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,F: The boy's experience with the computer is u...
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,F: The weather outside is not audible.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,F: The girl is happy and dissatisfied with the...
...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The boy with the hammer gets back up.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the axe manages to hit the boy w...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","After being hit by the axe, the boy with the ..."
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The girls are playing.


In [342]:
df.to_csv('/home/dtesta/MyProjects/Benchmark_PilotTest/DF_pilot_withCaptionFoils.csv')

In [2]:
def create_captionGPT(question, answer):
    response = openai.ChatCompletion.create(
        engine= deployment_name, # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
        messages=[
            {"role": "system", "content": f"You are an assistant designed to create declarative sentences." 
            f"Users will paste in a string of text a question Q and an answer A which are related to a video scene. Based on Q and A, you have to create "
            f"a sentence that is the caption for the video. Remember: the caption is a declarative sentence but it must be short and can contain only what is said in Q and A!"},
            {"role": "user", "content": f"Q: {question}\nA: {answer}"}
        ]
    )
    return response.choices[0].message.content

In [4]:
#load the df daaa cui prendere la caption e poi ci sovrascrivo i nuovi foil
df = pd.read_csv('/home/dtesta/MyProjects/Benchmark_PilotTest/DF_pilot.csv', index_col = 0)
df

,ID_Video,Category,Annotator,Question,Answer
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied
...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.


In [7]:
df.iloc[335,3]

'I do not have enough elements to answer this question.'

In [25]:
r = create_captionGPT(df.iloc[335,3], df.iloc[335,4])
r

'Based on A, I cannot create a caption for the video scene as there is not enough information to determine what the video is about. Please provide more details or context about the video.'

In [51]:


df = pd.read_csv('/home/dtesta/MyProjects/Benchmark_PilotTest/DF_pilot_withCaptionFoils.csv', index_col = 0)
df1 = pd.read_csv('/home/dtesta/MyProjects/Benchmark_PilotTest/DF_pilot_withCaption_Foil_Idefics.csv', index_col = 0)

In [53]:
df[('Foil')]

0       The video shows the girl using a phone to tak...
1       The people can protect themselves during an e...
2       The boy's experience with the computer is unc...
3                    The weather outside is not audible.
4       The girl is happy and dissatisfied with the w...
                             ...                        
339                The boy with the hammer gets back up.
340     The boy with the axe manages to hit the boy w...
341     After being hit by the axe, the boy with the ...
342                               The girls are playing.
343                       These guys are likely enemies.
Name: Foil, Length: 343, dtype: object

In [57]:
df1.to_csv('/home/dtesta/MyProjects/Benchmark_PilotTest/DF_pilot_withCaption_Foil_Idefics.csv')

In [56]:
df1[('Foil')]

0       The video shows the girl using a phone to tak...
1       The people can protect themselves during an e...
2       The boy's experience with the computer is unc...
3                    The weather outside is not audible.
4       The girl is happy and dissatisfied with the w...
                             ...                        
339                The boy with the hammer gets back up.
340     The boy with the axe manages to hit the boy w...
341     After being hit by the axe, the boy with the ...
342                               The girls are playing.
343                       These guys are likely enemies.
Name: Foil, Length: 343, dtype: object

In [55]:
df1[('Foil')] = df[('Foil')]


In [72]:
df = pd.read_csv('DF_pilot_withCaption_Foil_Idefics.csv', index_col = 0)
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil,Idefics_Answer
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,The video shows the girl using a phone to tak...,"No, the girl is not texting a message with her..."
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,The people can protect themselves during an e...,"If there was an earthquake, the computer monit..."
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,The boy's experience with the computer is unc...,"No, the girl is not using the computer for the..."
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,The weather outside is not audible.,The weather is sunny.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,The girl is happy and dissatisfied with the w...,The girl looked at the woman in disbelief.
...,...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The boy with the hammer gets back up.,The boy with the axe falls to the ground.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the axe manages to hit the boy w...,The boy with the sword manages to hit the boy ...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","After being hit by the axe, the boy with the ...",The boy with the sword and the boy with the ax...
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The girls are playing.,"Yes, the guys are playing."


In [75]:
df.iloc[335,3], df.iloc[335,4], df.iloc[335,5], df.iloc[335,6], df.iloc[335,7]

('Are the boys rehearsing for a historical reenactment?',
 "It's difficult to say.",
 "The boys' activities are uncertain.",
 "The girls' activities are uncertain.",
 'Yes, the boys are rehearsing for a historical reenactment.')